<a href="https://colab.research.google.com/github/YifanSong348/STATS302/blob/main/Week%204/OJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#STATS302 HW 4 - Yifan Song
###Problem 3. SVM
[ISL] 9.7 - 8

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
import io

import sklearn.linear_model as skl_lm
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn import ensemble
from sklearn import svm

from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error, accuracy_score
from sklearn.svm import SVC

import statsmodels.api as sm
from statsmodels.api import OLS

import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# import dataset
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['OJ.csv']))
print(df.head())

Saving OJ.csv to OJ.csv
   Unnamed: 0 Purchase  WeekofPurchase  ...  PctDiscCH  ListPriceDiff  STORE
0           1       CH             237  ...   0.000000           0.24      1
1           2       CH             239  ...   0.000000           0.24      1
2           3       CH             245  ...   0.091398           0.23      1
3           4       MM             227  ...   0.000000           0.00      1
4           5       CH             228  ...   0.000000           0.00      0

[5 rows x 19 columns]


In [3]:
df = df.drop(df.columns[0], axis=1)
print(df.shape)
print(df.head())
print(df.columns)
print(df.dtypes)

(1070, 18)
  Purchase  WeekofPurchase  StoreID  ...  PctDiscCH  ListPriceDiff  STORE
0       CH             237        1  ...   0.000000           0.24      1
1       CH             239        1  ...   0.000000           0.24      1
2       CH             245        1  ...   0.091398           0.23      1
3       MM             227        1  ...   0.000000           0.00      1
4       CH             228        7  ...   0.000000           0.00      0

[5 rows x 18 columns]
Index(['Purchase', 'WeekofPurchase', 'StoreID', 'PriceCH', 'PriceMM', 'DiscCH',
       'DiscMM', 'SpecialCH', 'SpecialMM', 'LoyalCH', 'SalePriceMM',
       'SalePriceCH', 'PriceDiff', 'Store7', 'PctDiscMM', 'PctDiscCH',
       'ListPriceDiff', 'STORE'],
      dtype='object')
Purchase           object
WeekofPurchase      int64
StoreID             int64
PriceCH           float64
PriceMM           float64
DiscCH            float64
DiscMM            float64
SpecialCH           int64
SpecialMM           int64
LoyalCH     

# a. Create a training set containing a random sample of 800 observations, and a test set containing the remaining observations.

In [4]:
np.random.seed(1)
df['Store7'] = df['Store7'].map({'Yes': 1, 'No': 0})
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Purchase'], axis=1),
                          df[['Purchase']], train_size=800)
print(X_train.shape)
print(X_test.shape)

(800, 17)
(270, 17)


# b. Fit a support vector classifier to the training data using cost=0.01, with Purchase as the response and the other variables as predictors. Use the summary() function to produce summary statistics, and describe the results obtained.

In [5]:
clf = svm.SVC(kernel='linear', C=0.01)
model = clf.fit(X_train, y_train.values.ravel())
print(model)
print("Number of support vectors: ", len(clf.support_vectors_))

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Number of support vectors:  611


In [6]:
model.n_support_

array([307, 304], dtype=int32)

Support vector classifier creates 611 support vectors out of 800 training points. Out of these, 307 belong to level MM and remaining 304 belong to level CH.

# c. What are the training and test error rates?

In [7]:
print("Training Error rate is: " + str(1 - accuracy_score(clf.predict(X_train), y_train)))
print("Test Error rate is: " + str(1 - accuracy_score(clf.predict(X_test), y_test)))

Training Error rate is: 0.31000000000000005
Test Error rate is: 0.3592592592592593


# d. Use the tune() function to select an optimal cost. Consider values in the range 0.01 to 10.

In [8]:
parameters = {'C':[0.01, 0.05, 0.1, 0.5, 1, 2, 3, 5, 7, 8, 9, 10]}
clf = GridSearchCV(SVC(random_state=1, kernel='linear'), parameters, cv=10)
clf.fit(X=X_train, y=y_train.values.ravel())
clf.best_estimator_

SVC(C=8, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

# e. Compute the training and test error rates using this new value for cost.

In [9]:
model = clf.best_estimator_
print("Training Error rate is: " + str(1 - accuracy_score(model.predict(X_train), y_train)))
print("Test Error rate is: " + str(1 - accuracy_score(model.predict(X_test), y_test)))

Training Error rate is: 0.16374999999999995
Test Error rate is: 0.1777777777777778


# f.  Repeat parts (b) through (e) using a support vector machine with a radial kernel. Use the default value for gamma.



> (b) and (c): c = 0.01



In [10]:
clf = svm.SVC(kernel='rbf', gamma='scale', C=0.01)
model = clf.fit(X_train, y_train.values.ravel())
print(model)
print("Number of support vectors: ", len(clf.support_vectors_))
print(model.n_support_)
print("Training Error rate is: " + str(1 - accuracy_score(clf.predict(X_train), y_train)))
print("Test Error rate is: " + str(1 - accuracy_score(clf.predict(X_test), y_test)))


SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Number of support vectors:  608
[304 304]
Training Error rate is: 0.38
Test Error rate is: 0.4185185185185185




> (d) and (e): changing c



In [11]:
clf = GridSearchCV(SVC(random_state=1, kernel='rbf'), parameters, n_jobs=4, cv=10)
clf.fit(X=X_train, y=y_train.values.ravel())
print(clf.best_estimator_)

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)


In [12]:
model = clf.best_estimator_
print("Training Error rate is: " + str(1 - accuracy_score(model.predict(X_train), y_train)))
print("Test Error rate is: " + str(1 - accuracy_score(model.predict(X_test), y_test)))

Training Error rate is: 0.38
Test Error rate is: 0.4185185185185185


# g. Repeat parts (b) through (e) using a support vector machine with a polynomial kernel. Set degree=2.



> (b) and (c): c = 0.01



In [13]:
clf = svm.SVC(kernel='poly', degree=2, C=0.01)
model = clf.fit(X_train, y_train.values.ravel())
print(model)
print("Number of support vectors: ", len(model.support_vectors_))
print(model.n_support_)
print("Training Error rate is: " + str(1 - accuracy_score(clf.predict(X_train), y_train)))
print("Test Error rate is: " + str(1 - accuracy_score(clf.predict(X_test), y_test)))

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Number of support vectors:  608
[304 304]
Training Error rate is: 0.38
Test Error rate is: 0.4185185185185185




> (d) and (e): changing c



In [14]:
clf = GridSearchCV(SVC(random_state=1, degree=2, kernel='poly'), parameters, n_jobs=4, cv=10)
clf.fit(X=X_train, y=y_train.values.ravel())
print(clf.best_estimator_)
model = clf.best_estimator_
print("Training Error rate is: " + str(1 - accuracy_score(model.predict(X_train), y_train)))
print("Test Error rate is: " + str(1 - accuracy_score(model.predict(X_test), y_test)))

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)
Training Error rate is: 0.38
Test Error rate is: 0.4185185185185185


# h.  Overall, which approach seems to give the best results on this data?

Overall, SVM with a linear kernel with cost = 8 produces minimum misclassification error on both train and test data.